In [23]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# Load preprocessed dataset
df = pd.read_csv('preprocessed_electric_vehicle_trip_data.csv')

# Define full feature list used in training
full_feature_list = [
    "acceleration(m/s²)", "speed(m/s)", "speedFactor",
    "totalEnergyConsumed(Wh)", "totalEnergyRegrained(Wh)",
    "slope(º)", "completedDistance(km)", "remainingRange(km)",
    "energyConsumptionRate(Wh/km)", "batteryVoltage(V)",
    "batteryCurrent(A)", "motorPower(W)", "motorTorque(Nm)",
    "ambientTemperature(ºC)", "motorTemperature(ºC)", "batteryTemperature(ºC)"
]

# Ensure all required features are present in dataset
missing_features = [feat for feat in full_feature_list if feat not in df.columns]
for feat in missing_features:
    df[feat] = 0  # Fill missing features with zeros or another default value

# Define fuzzy logic system
speed = ctrl.Antecedent(np.arange(0, 31, 1), 'speed')  # Speed range: 0 to 30 m/s
acceleration = ctrl.Antecedent(np.arange(-5, 6, 1), 'acceleration')  # Acceleration range: -5 to 5 m/s²
slope = ctrl.Antecedent(np.arange(-10, 11, 1), 'slope')  # Slope range: -10 to 10 degrees
soc = ctrl.Consequent(np.arange(0, 101, 1), 'soc')  # SoC range: 0 to 100%

# Define fuzzy membership functions
speed.automf(3, names=['low', 'medium', 'high'])  # 3 membership functions for speed
acceleration.automf(3, names=['low', 'medium', 'high'])  # 3 membership functions for acceleration
slope.automf(3, names=['low', 'medium', 'high'])  # 3 membership functions for slope
soc['low'] = fuzz.trimf(soc.universe, [0, 0, 50])
soc['medium'] = fuzz.trimf(soc.universe, [30, 50, 70])
soc['high'] = fuzz.trimf(soc.universe, [50, 100, 100])

# Define fuzzy rules
rule1 = ctrl.Rule(speed['low'] & acceleration['low'] & slope['low'], soc['low'])
rule2 = ctrl.Rule(speed['medium'] & acceleration['medium'] & slope['medium'], soc['medium'])
rule3 = ctrl.Rule(speed['high'] & acceleration['high'] & slope['high'], soc['high'])

# Create control system and simulation
soc_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
soc_simulation = ctrl.ControlSystemSimulation(soc_ctrl)

# Apply fuzzy model predictions
fuzzy_predictions = []
for i in range(len(df)):
    # Set input values
    speed_value = df.loc[i, 'speed(m/s)']
    acceleration_value = df.loc[i, 'acceleration(m/s²)']
    slope_value = df.loc[i, 'slope(º)']
    
    # Check if input values are within the defined ranges
    if not (0 <= speed_value <= 30):
        print(f"Warning: Speed value {speed_value} is out of range (0-30 m/s). Skipping this row.")
        fuzzy_predictions.append(np.nan)  # Append NaN for invalid inputs
        continue
    if not (-5 <= acceleration_value <= 5):
        print(f"Warning: Acceleration value {acceleration_value} is out of range (-5 to 5 m/s²). Skipping this row.")
        fuzzy_predictions.append(np.nan)  # Append NaN for invalid inputs
        continue
    if not (-10 <= slope_value <= 10):
        print(f"Warning: Slope value {slope_value} is out of range (-10 to 10 degrees). Skipping this row.")
        fuzzy_predictions.append(np.nan)  # Append NaN for invalid inputs
        continue
    
    # Set inputs and compute
    soc_simulation.input['speed'] = speed_value
    soc_simulation.input['acceleration'] = acceleration_value
    soc_simulation.input['slope'] = slope_value
    soc_simulation.compute()
    
    # Append the computed output
    fuzzy_predictions.append(soc_simulation.output['soc'])

# Add predictions to the dataframe
df['Fuzzy_Pred_SoC'] = fuzzy_predictions

# Compute regression metrics
def evaluate_regression(y_true, y_pred, model_name):
    return {
        'Model': model_name,
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R^2': r2_score(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

# Drop rows with NaN predictions before evaluation
df_clean = df.dropna(subset=['Fuzzy_Pred_SoC'])
metrics_fuzzy = evaluate_regression(df_clean['SoC(%)'], df_clean['Fuzzy_Pred_SoC'], 'Fuzzy Logic')

# Visualization
metrics_df = pd.DataFrame([metrics_fuzzy])
metrics_df.plot(kind='bar', figsize=(8, 5))
plt.title("Fuzzy Model Performance")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(loc='lower right')
plt.show()

KeyError: 'soc'